In [1]:
import sys
# appending a path
sys.path.append('../')

import numpy as np
import torch
import pychop
from pychop.chop import Chop
from pychop import float_params
from time import time
from numpy import linalg
import jax
from pychop import Customs
# from pychop.chop import chop
# from pychop.quant import quant
from time import time
# from scipy.io import savemat
# np.set_printoptions(precision=19)

In [2]:
np.random.seed(0)

X_np = np.random.randn(5000, 5000) # Numpy array
X_th = torch.Tensor(X_np) # torch array
X_jx = jax.numpy.asarray(X_np)
print(X_np)

#savemat("tests/verify.mat", {"X":X_np})

[[ 1.76405235  0.40015721  0.97873798 ...  0.92918181  0.22941801
   0.41440588]
 [ 0.30972382 -0.73745619 -1.53691988 ...  0.51687218 -0.03292069
   1.29811143]
 [-0.20211703 -0.833231    1.73360025 ...  0.75309415 -0.58103281
  -0.19837974]
 ...
 [ 1.07432182  1.188486    0.5092741  ...  0.07053449  0.59975911
  -2.41029925]
 [ 0.32432475 -0.02337844  1.62873399 ... -0.16088168 -1.59772992
   1.414703  ]
 [ 0.63460807  1.38090977  0.54829109 ...  0.30762729 -0.11078251
   0.83859307]]


In [3]:
X_th[0:5, 0]

tensor([ 1.7641,  0.3097, -0.2021,  2.4700,  0.3300])

### print unit-roundoff in machine

In [4]:
float_params()

,,u,xmins,xmin,xmax,p,emins,emin,emax
0,q43,6.25e-02,1.95e-03,1.56e-02,2.40e+02,4,-9,-6,7
1,q52,1.25e-01,1.53e-05,6.10e-05,5.73e+04,3,-16,-14,15
2,b,3.91e-03,9.18e-41,1.18e-38,3.39e+38,8,-133,-126,127
3,h,4.88e-04,5.96e-08,6.10e-05,6.55e+04,11,-24,-14,15
4,t,4.88e-04,1.15e-41,1.18e-38,3.40e+38,11,-136,-126,127
5,s,5.96e-08,1.40e-45,1.18e-38,3.40e+38,24,-149,-126,127
6,d,1.11e-16,4.94e-324,2.23e-308,1.80e+308,53,-1074,-1022,1023
7,q,9.63e-35,0.00e+00,0.00e+00,inf,113,-16494,-16382,16383


### set backend

In [5]:
# pychop.backend('torch')
pychop.backend('numpy', 1) # print information, NumPy is the default option.

Load NumPy backend.


### run chop

In [6]:
pyq_f = Chop('h')
st = time()
X_bit = pyq_f(X_np)
print("runtime:", time() - st)
print(X_bit[:10, 0])

runtime: 2.2011141777038574
[ 1.76367188  0.30981445 -0.20214844  2.47070312  0.33007812  0.30200195
  0.37133789  0.38720703 -1.93945312  0.56152344]


In [7]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=1)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:10, 0])

Load Troch backend.
runtime: 2.7147562503814697
tensor([ 1.7637,  0.3098, -0.2021,  2.4707,  0.3301,  0.3020,  0.3713,  0.3872,
        -1.9395,  0.5615])


In [8]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=2)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:10, 0])

Load Troch backend.
runtime: 1.5571949481964111
tensor([ 1.7646,  0.3098, -0.2020,  2.4707,  0.3301,  0.3022,  0.3713,  0.3875,
        -1.9395,  0.5620])


In [9]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=3)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:10, 0])

Load Troch backend.
runtime: 1.5467493534088135
tensor([ 1.7637,  0.3096, -0.2021,  2.4688,  0.3298,  0.3020,  0.3711,  0.3872,
        -1.9404,  0.5615])


In [10]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=4)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:10, 0])

Load Troch backend.
runtime: 1.6779279708862305
tensor([ 1.7637,  0.3096, -0.2020,  2.4688,  0.3298,  0.3020,  0.3711,  0.3872,
        -1.9395,  0.5615])


In [11]:
pychop.backend('torch', 1) # print information

Load Troch backend.


In [12]:
from pychop.lightchop import LightChop # only can be imported after using torch backend


values = torch.tensor([1.7641, 0.3097, -0.2021, 2.4700, 0.3300])


rounding_modes = ["nearest", "up", "down", "towards_zero", 
                 "stochastic_equal", "stochastic_proportional"]

# Compare with PyTorch's native FP16
fp16_native = values.to(dtype=torch.float16).to(dtype=torch.float32)

print("Input values:      ", values)
print("PyTorch FP16:      ", fp16_native)
print()

print()
rounding_modes_num = [1, 2, 3, 4, 5, 6]

for mode in rounding_modes_num[:4]:
    pyq_f = Chop('h', rmode=mode)
    groud_truth = pyq_f(values)
    
    # half precision simulator (5 exponent bits, 10 mantissa bits)
    fp16_sim = LightChop(5, 10, rmode=mode)
    emulated = fp16_sim.quantize(values)
    assert np.array_equal(emulated, groud_truth), print("error rmode 3")
    
    print(f"{rounding_modes[mode-1]:12}, ", "Correct:", f"   {emulated}")
    print(f"{rounding_modes[mode-1]:12}, ", "Emulated:", f"{groud_truth}")



Input values:       tensor([ 1.7641,  0.3097, -0.2021,  2.4700,  0.3300])
PyTorch FP16:       tensor([ 1.7637,  0.3098, -0.2021,  2.4707,  0.3301])


nearest     ,  Correct:    tensor([ 1.7637,  0.3098, -0.2021,  2.4707,  0.3301])
nearest     ,  Emulated: tensor([ 1.7637,  0.3098, -0.2021,  2.4707,  0.3301])
up          ,  Correct:    tensor([ 1.7646,  0.3098, -0.2020,  2.4707,  0.3301])
up          ,  Emulated: tensor([ 1.7646,  0.3098, -0.2020,  2.4707,  0.3301])
down        ,  Correct:    tensor([ 1.7637,  0.3096, -0.2021,  2.4688,  0.3298])
down        ,  Emulated: tensor([ 1.7637,  0.3096, -0.2021,  2.4688,  0.3298])
towards_zero,  Correct:    tensor([ 1.7637,  0.3096, -0.2020,  2.4688,  0.3298])
towards_zero,  Emulated: tensor([ 1.7637,  0.3096, -0.2020,  2.4688,  0.3298])


In [13]:
from pychop.layers import QuantizedLayer

formats = {
    "fp32": (8, 23),    # Standard IEEE 754 float32
    "fp16": (5, 10),    # Standard IEEE 754 float16
    "bf16": (8, 7),     # bfloat16
    "fp8": (5, 2),      # Example 8-bit float
}

# Test different rounding modes
rounding_modes = [
    "nearest",
    "plus_inf",
    "minus_inf",
    "towards_zero",
    "stoc_equal",
    "stoc_prop"
]
# Create a sample tensor
x = torch.tensor([0.1, 0.3, 1.7, 3.9, -2.5])

# Test quantization
mp = LightChop(*formats["bf16"], mode)

print("Original values:", x)
for mode in rounding_modes:
    result = mp.quantize(x)
    print(f"{mode}:", result)

# Test with a layer
layer = QuantizedLayer(*formats["bf16"])
input_tensor = torch.randn(3, 4)
output = layer(input_tensor)

Original values: tensor([ 0.1000,  0.3000,  1.7000,  3.9000, -2.5000])
nearest: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])
plus_inf: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])
minus_inf: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])
towards_zero: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])
stoc_equal: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])
stoc_prop: tensor([ 0.0996,  0.2988,  1.6953,  3.8906, -2.5000])


In [14]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=1)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:5, 0])

Load Troch backend.
runtime: 1.7165188789367676
tensor([ 1.7637,  0.3098, -0.2021,  2.4707,  0.3301])


In [15]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=2)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:5, 0])

Load Troch backend.
runtime: 1.4376509189605713
tensor([ 1.7646,  0.3098, -0.2020,  2.4707,  0.3301])


In [16]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=3)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:5, 0])

Load Troch backend.
runtime: 1.4878880977630615
tensor([ 1.7637,  0.3096, -0.2021,  2.4688,  0.3298])


In [17]:
pychop.backend('torch', 1) # print information
pyq_f = Chop('h', rmode=4)
st = time()
X_bit = pyq_f(X_th)
print("runtime:", time() - st)
print(X_bit[:5, 0])

Load Troch backend.
runtime: 1.7052178382873535
tensor([ 1.7637,  0.3096, -0.2020,  2.4688,  0.3298])


In [18]:
pychop.backend('jax', 1) # print information
pyq_f = Chop('h')
st = time()
X_bit = pyq_f(X_jx)
print("runtime:", time() - st)
print(X_bit)

Load JAX backend.
runtime: 12.719720125198364
[[ 1.7636719   0.40014648  0.9785156  ...  0.9291992   0.22937012
   0.41430664]
 [ 0.30981445 -0.7373047  -1.5371094  ...  0.51708984 -0.03292847
   1.2978516 ]
 [-0.20214844 -0.8330078   1.7333984  ...  0.7529297  -0.5810547
  -0.19836426]
 ...
 [ 1.0742188   1.1884766   0.50927734 ...  0.07055664  0.5996094
  -2.4101562 ]
 [ 0.32421875 -0.02337646  1.6289062  ... -0.16088867 -1.5976562
   1.4150391 ]
 [ 0.6347656   1.3808594   0.54833984 ...  0.3076172  -0.11077881
   0.8383789 ]]


### Custom precision

In [19]:
# pychop.backend('torch')
pychop.backend('numpy', 1) # print information, NumPy is the default option.
ct1 = Customs(emax=15, t=11) # half precision,  t is the number of bits in the significand 
                             # (including the hidden bit) and emax is the maximum value of the exponent

pyq_f = Chop(customs=ct1, rmode=3) 
X_bit = pyq_f(X_np)
print(X_bit[:10, 0])

ct2 = Customs(exp_bits=5, sig_bits=10) # half precision (5 exponent bits, 10+1 significand bits, 1 is implicit bits)
pyq_f = Chop(customs=ct2, rmode=3)
X_bit = pyq_f(X_np)
print(X_bit[:10, 0])

Load NumPy backend.
[ 1.76367188  0.30957031 -0.20214844  2.46875     0.32983398  0.30200195
  0.37109375  0.38720703 -1.94042969  0.56152344]
[ 1.76367188  0.30957031 -0.20214844  2.46875     0.32983398  0.30200195
  0.37109375  0.38720703 -1.94042969  0.56152344]


In [20]:
ct1 = Customs(7, 4) # q43 precision,  t is the number of bits in the significand 
                             # (including the hidden bit) and emax is the maximum value of the exponent

pyq_f = Chop(customs=ct1, rmode=3) 
X_bit = pyq_f(X_np)
print(X_bit[:10, 0])

ct2 = Customs(exp_bits=4, sig_bits=3) # q43 precision (4 exponent bits, 3+1 significand bits, 1 is implicit bits)
pyq_f = Chop(customs=ct2, rmode=3)
X_bit = pyq_f(X_np)
print(X_bit[:10, 0])

[ 1.75      0.28125  -0.203125  2.25      0.3125    0.28125   0.34375
  0.375    -2.        0.5     ]
[ 1.75      0.28125  -0.203125  2.25      0.3125    0.28125   0.34375
  0.375    -2.        0.5     ]


### integer quantization

In [21]:
pychop.backend('numpy')
pyq_f = pychop.Chopi(16)
X_q = pyq_f.quantize(X_np)
X_inv = pyq_f.dequantize(X_q)
linalg.norm(X_inv - X_np)

0.48872269380767824

In [22]:
pychop.backend('torch')
pyq_f = pychop.Chopi(num_bits=16)
X_q = pyq_f.quantize(X_th)
X_inv = pyq_f.dequantize(X_q)
linalg.norm(X_inv - X_th)

0.48856017

In [23]:
X_q.dtype

torch.int16

In [24]:
pychop.backend('jax')
pyq_f = pychop.Chopi(num_bits=2)
X_q = pyq_f.quantize(X_jx)
X_inv = pyq_f.dequantize(X_q)
linalg.norm(X_inv - X_jx)

/Users/chenxinye/pychop/examples/../pychop/jx/integer.py:44: UserWarning: Current int type does not support this bitwidth, use int64 to simulate.
  warnings.warn("Current int type does not support this bitwidth, use int64 to simulate.")
/opt/anaconda3/lib/python3.12/site-packages/jax/_src/numpy/array_methods.py:122: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


4995.702

### fixed point quantization

In [25]:
pychop.backend('numpy')
from pychop import Chopf

pyq_f = Chopf(ibits=4, fbits=4)

pyq_f(X_np)

array([[ 1.75  ,  0.375 ,  1.    , ...,  0.9375,  0.25  ,  0.4375],
       [ 0.3125, -0.75  , -1.5625, ...,  0.5   , -0.0625,  1.3125],
       [-0.1875, -0.8125,  1.75  , ...,  0.75  , -0.5625, -0.1875],
       ...,
       [ 1.0625,  1.1875,  0.5   , ...,  0.0625,  0.625 , -2.4375],
       [ 0.3125, -0.    ,  1.625 , ..., -0.1875, -1.625 ,  1.4375],
       [ 0.625 ,  1.375 ,  0.5625, ...,  0.3125, -0.125 ,  0.8125]])

In [26]:
pychop.backend('torch')
pyq_f = Chopf() # default: ibits=4, fbits=4,
pyq_f(X_th)

tensor([[ 1.7500,  0.3750,  1.0000,  ...,  0.9375,  0.2500,  0.4375],
        [ 0.3125, -0.7500, -1.5625,  ...,  0.5000, -0.0625,  1.3125],
        [-0.1875, -0.8125,  1.7500,  ...,  0.7500, -0.5625, -0.1875],
        ...,
        [ 1.0625,  1.1875,  0.5000,  ...,  0.0625,  0.6250, -2.4375],
        [ 0.3125, -0.0000,  1.6250,  ..., -0.1875, -1.6250,  1.4375],
        [ 0.6250,  1.3750,  0.5625,  ...,  0.3125, -0.1250,  0.8125]])

In [27]:
pychop.backend('jax')
pyq_f = Chopf()
pyq_f(X_jx)

Array([[ 1.75  ,  0.375 ,  1.    , ...,  0.9375,  0.25  ,  0.4375],
       [ 0.3125, -0.75  , -1.5625, ...,  0.5   , -0.0625,  1.3125],
       [-0.1875, -0.8125,  1.75  , ...,  0.75  , -0.5625, -0.1875],
       ...,
       [ 1.0625,  1.1875,  0.5   , ...,  0.0625,  0.625 , -2.4375],
       [ 0.3125, -0.    ,  1.625 , ..., -0.1875, -1.625 ,  1.4375],
       [ 0.625 ,  1.375 ,  0.5625, ...,  0.3125, -0.125 ,  0.8125]],      dtype=float32)

In [28]:
pyq_f.rmode

1

In [29]:
import numpy as np
import torch
import pychop
from numpy import linalg
import jax

X_np = np.random.randn(500, 500) # NumPy array
X_th = torch.Tensor(X_np) # Torch array
X_jx = jax.numpy.asarray(X_np) # JAX array
print(X_np)

pychop.backend('numpy')
pyq_f = pychop.Chopi(num_bits=8) # The larger the ``bits`` are, the more accurate of the reconstruction is 
X_q = pyq_f.quantize(X_np) # quant array -> integer
X_inv = pyq_f.dequantize(X_q) # dequant array -> floating point values
linalg.norm(X_inv - X_np)
    

pychop.backend('torch')
pyq_f = pychop.Chopi(num_bits=8)
X_q = pyq_f.quantize(X_th)  # quant array -> integer
X_inv = pyq_f.dequantize(X_q) # dequant array -> floating point values
linalg.norm(X_inv - X_np)


pychop.backend('jax')
pyq_f = pychop.Chopi(num_bits=8)
X_q = pyq_f.quantize(X_jx) # quant array -> integer
X_inv = pyq_f.dequantize(X_q) # dequant array -> floating point values 
linalg.norm(X_inv - X_jx)

[[ 1.76405235  0.40015721  0.97873798 ... -0.51423397 -1.01804188
  -0.07785476]
 [ 0.38273243 -0.03424228  1.09634685 ...  0.0941923  -1.14761094
  -0.35811408]
 [ 0.55596268  0.89247389 -0.42231482 ... -0.24679356 -1.07934317
  -0.11422555]
 ...
 [-1.28277909 -0.17950901  0.27819164 ... -1.31869965 -0.22116722
   0.41593528]
 [-1.23162889 -0.51221521  0.12511042 ...  0.84961987 -0.81843243
   0.16746637]
 [-1.09316138 -0.74909675  1.11398219 ... -0.52799973 -0.65857377
  -0.55711525]]


10.378907